# Comparison with Stata

For potential users coming from [Stata](https://en.wikipedia.org/wiki/Stata)
this page is meant to demonstrate how different Stata operations would be
performed in pandas.

If you’re new to pandas, you might want to first read through [10 Minutes to pandas](../04_user_guide/10min.ipynb#min)
to familiarize yourself with the library.

As is customary, we import pandas and NumPy as follows:

In [1]:
import pandas as pd
import numpy as np

## Data structures

### General terminology translation

|pandas|Stata|
|:------------------:|:------------------:|
|DataFrame|data set|
|column|variable|
|row|observation|
|groupby|bysort|
|NaN|.|

### `DataFrame`

A `DataFrame` in pandas is analogous to a Stata data set – a two-dimensional
data source with labeled columns that can be of different types. As will be
shown in this document, almost any operation that can be applied to a data set
in Stata can also be accomplished in pandas.

### `Series`

A `Series` is the data structure that represents one column of a
`DataFrame`. Stata doesn’t have a separate data structure for a single column,
but in general, working with a `Series` is analogous to referencing a column
of a data set in Stata.

### `Index`

Every `DataFrame` and `Series` has an `Index` – labels on the
*rows* of the data. Stata does not have an exactly analogous concept. In Stata, a data set’s
rows are essentially unlabeled, other than an implicit integer index that can be
accessed with `_n`.

In pandas, if no index is specified, an integer index is also used by default
(first row = 0, second row = 1, and so on). While using a labeled `Index` or
`MultiIndex` can enable sophisticated analyses and is ultimately an important
part of pandas to understand, for this comparison we will essentially ignore the
`Index` and just treat the `DataFrame` as a collection of columns. Please
see the indexing documentation for much more on how to use an
`Index` effectively.

### Copies vs. in place operations

Most pandas operations return copies of the `Series`/`DataFrame`. To make the changes “stick”,
you’ll need to either assign to a new variable:

> sorted_df = df.sort_values(“col1”)


or overwrite the original one:

>df = df.sort_values("col1")

>**Note**
>
>You will see an `inplace=True` keyword argument available for some methods:

>df.sort_values("col1", inplace=True)

Its use is discouraged. :ref:`More information. <indexing.view_versus_copy>`

## Data input / output

### Constructing a DataFrame from values

A Stata data set can be built from specified values by
placing the data after an `input` statement and
specifying the column names.

```stata
input x y
1 2
3 4
5 6
end
```


A pandas `DataFrame` can be constructed in many different ways,
but for a small number of values, it is often convenient to specify it as
a Python dictionary, where the keys are the column names
and the values are the data.

In [4]:
df = pd.DataFrame({"x": [1, 3, 5], "y": [2, 4, 6]})
df

,x,y
0,1,2
1,3,4
2,5,6


### Reading external data

Like Stata, pandas provides utilities for reading in data from
many formats.  The `tips` data set, found within the pandas
tests ([csv](https://raw.github.com/pandas-dev/pandas/master/pandas/tests/io/data/csv/tips.csv))
will be used in many of the following examples.

Stata provides `import delimited` to read csv data into a data set in memory.
If the `tips.csv` file is in the current working directory, we can import it as follows.

```stata
import delimited tips.csv
```


The pandas method is `read_csv()`, which works similarly. Additionally, it will automatically download
the data set if presented with a url.

In [5]:
url = (
    "https://raw.github.com/pandas-dev"
    "/pandas/master/pandas/tests/io/data/csv/tips.csv"
)
tips = pd.read_csv(url)
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


Like `import delimited`, `read_csv()` can take a number of parameters to specify
how the data should be parsed.  For example, if the data were instead tab delimited,
did not have column names, and existed in the current working directory,
the pandas command would be:

```python
tips = pd.read_csv("tips.csv", sep="\t", header=None)

# alternatively, read_table is an alias to read_csv with tab delimiter
tips = pd.read_table("tips.csv", header=None)
```


pandas can also read Stata data sets in `.dta` format with the `read_stata()` function.

```python
df = pd.read_stata("data.dta")
```


In addition to text/csv and Stata files, pandas supports a variety of other data formats
such as Excel, SAS, HDF5, Parquet, and SQL databases.  These are all read via a `pd.read_*`
function.  See the IO documentation for more details.

### Limiting output

By default, pandas will truncate output of large `DataFrame`s to show the first and last rows.
This can be overridden by changing the pandas options, or using
`DataFrame.head()` or `DataFrame.tail()`.

In [6]:
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


The equivalent in Stata would be:

```stata
list in 1/5
```


### Exporting data

The inverse of `import delimited` in Stata is `export delimited`

```stata
export delimited tips2.csv
```


Similarly in pandas, the opposite of `read_csv` is `DataFrame.to_csv()`.

```python
tips.to_csv("tips2.csv")
```


pandas can also export to Stata file format with the `DataFrame.to_stata()` method.

```python
tips.to_stata("tips2.dta")
```


## Data operations

### Operations on columns

In Stata, arbitrary math expressions can be used with the `generate` and
`replace` commands on new or existing columns. The `drop` command drops
the column from the data set.

```stata
replace total_bill = total_bill - 2
generate new_bill = total_bill / 2
drop new_bill
```


pandas provides vectorized operations by specifying the individual `Series` in the
`DataFrame`. New columns can be assigned in the same way. The `DataFrame.drop()` method drops
a column from the `DataFrame`.

In [7]:
tips["total_bill"] = tips["total_bill"] - 2
tips["new_bill"] = tips["total_bill"] / 2
tips

tips = tips.drop("new_bill", axis=1)

### Filtering

Filtering in Stata is done with an `if` clause on one or more columns.

```stata
list if total_bill > 10
```


DataFrames can be filtered in multiple ways; the most intuitive of which is using
boolean indexing.

In [8]:
tips[tips["total_bill"] > 10]

,total_bill,tip,sex,smoker,day,time,size
0,14.99,1.01,Female,No,Sun,Dinner,2
2,19.01,3.50,Male,No,Sun,Dinner,3
3,21.68,3.31,Male,No,Sun,Dinner,2
4,22.59,3.61,Female,No,Sun,Dinner,4
5,23.29,4.71,Male,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3
240,25.18,2.00,Female,Yes,Sat,Dinner,2
241,20.67,2.00,Male,Yes,Sat,Dinner,2
242,15.82,1.75,Male,No,Sat,Dinner,2


The above statement is simply passing a `Series` of `True`/`False` objects to the DataFrame,
returning all rows with `True`.

In [9]:
is_dinner = tips["time"] == "Dinner"
is_dinner
is_dinner.value_counts()
tips[is_dinner]

,total_bill,tip,sex,smoker,day,time,size
0,14.99,1.01,Female,No,Sun,Dinner,2
1,8.34,1.66,Male,No,Sun,Dinner,3
2,19.01,3.50,Male,No,Sun,Dinner,3
3,21.68,3.31,Male,No,Sun,Dinner,2
4,22.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3
240,25.18,2.00,Female,Yes,Sat,Dinner,2
241,20.67,2.00,Male,Yes,Sat,Dinner,2
242,15.82,1.75,Male,No,Sat,Dinner,2


### If/then logic

In Stata, an `if` clause can also be used to create new columns.

```stata
generate bucket = "low" if total_bill < 10
replace bucket = "high" if total_bill >= 10
```


The same operation in pandas can be accomplished using
the `where` method from `numpy`.

In [10]:
tips["bucket"] = np.where(tips["total_bill"] < 10, "low", "high")
tips

,total_bill,tip,sex,smoker,day,time,size,bucket
0,14.99,1.01,Female,No,Sun,Dinner,2,high
1,8.34,1.66,Male,No,Sun,Dinner,3,low
2,19.01,3.50,Male,No,Sun,Dinner,3,high
3,21.68,3.31,Male,No,Sun,Dinner,2,high
4,22.59,3.61,Female,No,Sun,Dinner,4,high
...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,high
240,25.18,2.00,Female,Yes,Sat,Dinner,2,high
241,20.67,2.00,Male,Yes,Sat,Dinner,2,high
242,15.82,1.75,Male,No,Sat,Dinner,2,high



<dl style='margin: 20px 0;'>
<dt>::</dt>
<dd>

<dl style='margin: 20px 0;'>
<dt>suppress</dt>
<dd>
</dd>

</dl>

tips = tips.drop(“bucket”, axis=1)

</dd>

</dl>

### Date functionality

Stata provides a variety of functions to do operations on
date/datetime columns.

```stata
generate date1 = mdy(1, 15, 2013)
generate date2 = date("Feb152015", "MDY")

generate date1_year = year(date1)
generate date2_month = month(date2)

* shift date to beginning of next month
generate date1_next = mdy(month(date1) + 1, 1, year(date1)) if month(date1) != 12
replace date1_next = mdy(1, 1, year(date1) + 1) if month(date1) == 12
generate months_between = mofd(date2) - mofd(date1)

list date1 date2 date1_year date2_month date1_next months_between
```


The equivalent pandas operations are shown below.  In addition to these
functions, pandas supports other Time Series features
not available in Stata (such as time zone handling and custom offsets) –
see the timeseries documentation for more details.

In [11]:
tips["date1"] = pd.Timestamp("2013-01-15")
tips["date2"] = pd.Timestamp("2015-02-15")
tips["date1_year"] = tips["date1"].dt.year
tips["date2_month"] = tips["date2"].dt.month
tips["date1_next"] = tips["date1"] + pd.offsets.MonthBegin()
tips["months_between"] = tips["date2"].dt.to_period("M") - tips[
    "date1"
].dt.to_period("M")

tips[
    ["date1", "date2", "date1_year", "date2_month", "date1_next", "months_between"]
]

,date1,date2,date1_year,date2_month,date1_next,months_between
0,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
1,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
2,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
3,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
4,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
...,...,...,...,...,...,...
239,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
240,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
241,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>
242,2013-01-15,2015-02-15,2013,2,2013-02-01,<25 * MonthEnds>


In [13]:
# cleanup
tips = tips.drop(["date1", "date2", "date1_year", "date2_month", "date1_next", "months_between"],
axis=1,
)


### Selection of columns

Stata provides keywords to select, drop, and rename columns.

```stata
keep sex total_bill tip

drop sex

rename total_bill total_bill_2
```


The same operations are expressed in pandas below.

#### Keep certain columns

In [14]:
tips[["sex", "total_bill", "tip"]]

,sex,total_bill,tip
0,Female,14.99,1.01
1,Male,8.34,1.66
2,Male,19.01,3.50
3,Male,21.68,3.31
4,Female,22.59,3.61
...,...,...,...
239,Male,27.03,5.92
240,Female,25.18,2.00
241,Male,20.67,2.00
242,Male,15.82,1.75


#### Drop a column

In [15]:
tips.drop("sex", axis=1)

,total_bill,tip,smoker,day,time,size,bucket
0,14.99,1.01,No,Sun,Dinner,2,high
1,8.34,1.66,No,Sun,Dinner,3,low
2,19.01,3.50,No,Sun,Dinner,3,high
3,21.68,3.31,No,Sun,Dinner,2,high
4,22.59,3.61,No,Sun,Dinner,4,high
...,...,...,...,...,...,...,...
239,27.03,5.92,No,Sat,Dinner,3,high
240,25.18,2.00,Yes,Sat,Dinner,2,high
241,20.67,2.00,Yes,Sat,Dinner,2,high
242,15.82,1.75,No,Sat,Dinner,2,high


#### Rename a column

In [16]:
tips.rename(columns={"total_bill": "total_bill_2"})

,total_bill_2,tip,sex,smoker,day,time,size,bucket
0,14.99,1.01,Female,No,Sun,Dinner,2,high
1,8.34,1.66,Male,No,Sun,Dinner,3,low
2,19.01,3.50,Male,No,Sun,Dinner,3,high
3,21.68,3.31,Male,No,Sun,Dinner,2,high
4,22.59,3.61,Female,No,Sun,Dinner,4,high
...,...,...,...,...,...,...,...,...
239,27.03,5.92,Male,No,Sat,Dinner,3,high
240,25.18,2.00,Female,Yes,Sat,Dinner,2,high
241,20.67,2.00,Male,Yes,Sat,Dinner,2,high
242,15.82,1.75,Male,No,Sat,Dinner,2,high


### Sorting by values

Sorting in Stata is accomplished via `sort`

```stata
sort sex total_bill
```


pandas has a `DataFrame.sort_values()` method, which takes a list of columns to sort by.

In [17]:
tips = tips.sort_values(["sex", "total_bill"])
tips

,total_bill,tip,sex,smoker,day,time,size,bucket
67,1.07,1.00,Female,Yes,Sat,Dinner,1,low
92,3.75,1.00,Female,Yes,Fri,Dinner,2,low
111,5.25,1.00,Female,No,Sat,Dinner,1,low
145,6.35,1.50,Female,No,Thur,Lunch,2,low
135,6.51,1.25,Female,No,Thur,Lunch,2,low
...,...,...,...,...,...,...,...,...
182,43.35,3.50,Male,Yes,Sun,Dinner,3,high
156,46.17,5.00,Male,No,Sun,Dinner,6,high
59,46.27,6.73,Male,No,Sat,Dinner,4,high
212,46.33,9.00,Male,No,Sat,Dinner,4,high


## String processing

### Finding length of string

Stata determines the length of a character string with the `strlen()` and
`ustrlen()` functions for ASCII and Unicode strings, respectively.

```stata
generate strlen_time = strlen(time)
generate ustrlen_time = ustrlen(time)
```


You can find the length of a character string with `Series.str.len()`.
In Python 3, all strings are Unicode strings. `len` includes trailing blanks.
Use `len` and `rstrip` to exclude trailing blanks.

In [18]:
tips["time"].str.len()
tips["time"].str.rstrip().str.len()

67     6
92     6
111    6
145    5
135    5
      ..
182    6
156    6
59     6
212    6
170    6
Name: time, Length: 244, dtype: int64

### Finding position of substring

Stata determines the position of a character in a string with the `strpos()` function.
This takes the string defined by the first argument and searches for the
first position of the substring you supply as the second argument.

```stata
generate str_position = strpos(sex, "ale")
```


You can find the position of a character in a column of strings with the `Series.str.find()`
method. `find` searches for the first position of the substring. If the substring is found, the
method returns its position. If not found, it returns `-1`. Keep in mind that Python indexes are
zero-based.

In [19]:
tips["sex"].str.find("ale")

67     3
92     3
111    3
145    3
135    3
      ..
182    1
156    1
59     1
212    1
170    1
Name: sex, Length: 244, dtype: int64

### Extracting substring by position

Stata extracts a substring from a string based on its position with the `substr()` function.

```stata
generate short_sex = substr(sex, 1, 1)
```


With pandas you can use `[]` notation to extract a substring
from a string by position locations. Keep in mind that Python
indexes are zero-based.

In [20]:
tips["sex"].str[0:1]

67     F
92     F
111    F
145    F
135    F
      ..
182    M
156    M
59     M
212    M
170    M
Name: sex, Length: 244, dtype: object

### Extracting nth word

The Stata `word()` function returns the nth word from a string.
The first argument is the string you want to parse and the
second argument specifies which word you want to extract.

```stata
clear
input str20 string
"John Smith"
"Jane Cook"
end

generate first_name = word(name, 1)
generate last_name = word(name, -1)
```


The simplest way to extract words in pandas is to split the strings by spaces, then reference the
word by index. Note there are more powerful approaches should you need them.

In [21]:
firstlast = pd.DataFrame({"String": ["John Smith", "Jane Cook"]})
firstlast["First_Name"] = firstlast["String"].str.split(" ", expand=True)[0]
firstlast["Last_Name"] = firstlast["String"].str.rsplit(" ", expand=True)[0]
firstlast

,String,First_Name,Last_Name
0,John Smith,John,John
1,Jane Cook,Jane,Jane


### Changing case

The Stata `strupper()`, `strlower()`, `strproper()`,
`ustrupper()`, `ustrlower()`, and `ustrtitle()` functions
change the case of ASCII and Unicode strings, respectively.

```stata
clear
input str20 string
"John Smith"
"Jane Cook"
end

generate upper = strupper(string)
generate lower = strlower(string)
generate title = strproper(string)
list
```


The equivalent pandas methods are `Series.str.upper()`, `Series.str.lower()`, and
`Series.str.title()`.

In [22]:
firstlast = pd.DataFrame({"string": ["John Smith", "Jane Cook"]})
firstlast["upper"] = firstlast["string"].str.upper()
firstlast["lower"] = firstlast["string"].str.lower()
firstlast["title"] = firstlast["string"].str.title()
firstlast

,string,upper,lower,title
0,John Smith,JOHN SMITH,john smith,John Smith
1,Jane Cook,JANE COOK,jane cook,Jane Cook


## Merging

The following tables will be used in the merge examples:

In [23]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})
df1
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})
df2

,key,value
0,B,-0.693131
1,D,1.676520
2,D,-0.571603
3,E,1.383426


In Stata, to perform a merge, one data set must be in memory
and the other must be referenced as a file name on disk. In
contrast, Python must have both `DataFrames` already in memory.

By default, Stata performs an outer join, where all observations
from both data sets are left in memory after the merge. One can
keep only observations from the initial data set, the merged data set,
or the intersection of the two by using the values created in the
`_merge` variable.

```stata
* First create df2 and save to disk
clear
input str1 key
B
D
D
E
end
generate value = rnormal()
save df2.dta

* Now create df1 in memory
clear
input str1 key
A
B
C
D
end
generate value = rnormal()

preserve

* Left join
merge 1:n key using df2.dta
keep if _merge == 1

* Right join
restore, preserve
merge 1:n key using df2.dta
keep if _merge == 2

* Inner join
restore, preserve
merge 1:n key using df2.dta
keep if _merge == 3

* Outer join
restore
merge 1:n key using df2.dta
```


pandas DataFrames have a `merge()` method, which provides similar functionality. The
data does not have to be sorted ahead of time, and different join types are accomplished via the
`how` keyword.

In [24]:
inner_join = df1.merge(df2, on=["key"], how="inner")
inner_join

left_join = df1.merge(df2, on=["key"], how="left")
left_join

right_join = df1.merge(df2, on=["key"], how="right")
right_join

outer_join = df1.merge(df2, on=["key"], how="outer")
outer_join

,key,value_x,value_y
0,A,0.840046,NaN
1,B,0.465596,-0.693131
2,C,-2.014986,NaN
3,D,-1.161376,1.676520
4,D,-1.161376,-0.571603
5,E,NaN,1.383426


## Missing data

Both pandas and Stata have a representation for missing data.

pandas represents missing data with the special float value `NaN` (not a number).  Many of the
semantics are the same; for example missing data propagates through numeric operations, and is
ignored by default for aggregations.

In [25]:
outer_join
outer_join["value_x"] + outer_join["value_y"]
outer_join["value_x"].sum()

-3.0320967657192646

One difference is that missing data cannot be compared to its sentinel value.
For example, in Stata you could do this to filter missing values.

```stata
* Keep missing values
list if value_x == .
* Keep non-missing values
list if value_x != .
```


In pandas, `Series.isna()` and `Series.notna()` can be used to filter the rows.

In [26]:
outer_join[outer_join["value_x"].isna()]
outer_join[outer_join["value_x"].notna()]

,key,value_x,value_y
0,A,0.840046,NaN
1,B,0.465596,-0.693131
2,C,-2.014986,NaN
3,D,-1.161376,1.676520
4,D,-1.161376,-0.571603


pandas provides a variety of methods to work with missing data. Here are some examples:

### Drop rows with missing values

In [27]:
outer_join.dropna()

,key,value_x,value_y
1,B,0.465596,-0.693131
3,D,-1.161376,1.676520
4,D,-1.161376,-0.571603


### Forward fill from previous rows

In [28]:
outer_join.fillna(method="ffill")

,key,value_x,value_y
0,A,0.840046,NaN
1,B,0.465596,-0.693131
2,C,-2.014986,-0.693131
3,D,-1.161376,1.676520
4,D,-1.161376,-0.571603
5,E,-1.161376,1.383426


### Replace missing values with a specified value

Using the mean:

In [29]:
outer_join["value_x"].fillna(outer_join["value_x"].mean())

0    0.840046
1    0.465596
2   -2.014986
3   -1.161376
4   -1.161376
5   -0.606419
Name: value_x, dtype: float64

## GroupBy

### Aggregation

Stata’s `collapse` can be used to group by one or
more key variables and compute aggregations on
numeric columns.

```stata
collapse (sum) total_bill tip, by(sex smoker)
```


pandas provides a flexible `groupby` mechanism that allows similar aggregations. See the
groupby documentation for more details and examples.

In [30]:
tips_summed = tips.groupby(["sex", "smoker"])[["total_bill", "tip"]].sum()
tips_summed

total_bill     tip
sex    smoker                    
Female No          869.68  149.77
       Yes         527.27   96.74
Male   No         1725.75  302.00
       Yes        1217.07  183.07

### Transformation

In Stata, if the group aggregations need to be used with the
original data set, one would usually use `bysort` with `egen()`.
For example, to subtract the mean for each observation by smoker group.

```stata
bysort sex smoker: egen group_bill = mean(total_bill)
generate adj_total_bill = total_bill - group_bill
```


pandas provides a [Transformation](../04_user_guide/36_groupby.ipynb#groupby-transform) mechanism that allows these type of operations to be
succinctly expressed in one operation.

In [31]:
gb = tips.groupby("smoker")["total_bill"]
tips["adj_total_bill"] = tips["total_bill"] - gb.transform("mean")
tips

,total_bill,tip,sex,smoker,day,time,size,bucket,adj_total_bill
67,1.07,1.00,Female,Yes,Sat,Dinner,1,low,-17.686344
92,3.75,1.00,Female,Yes,Fri,Dinner,2,low,-15.006344
111,5.25,1.00,Female,No,Sat,Dinner,1,low,-11.938278
145,6.35,1.50,Female,No,Thur,Lunch,2,low,-10.838278
135,6.51,1.25,Female,No,Thur,Lunch,2,low,-10.678278
...,...,...,...,...,...,...,...,...,...
182,43.35,3.50,Male,Yes,Sun,Dinner,3,high,24.593656
156,46.17,5.00,Male,No,Sun,Dinner,6,high,28.981722
59,46.27,6.73,Male,No,Sat,Dinner,4,high,29.081722
212,46.33,9.00,Male,No,Sat,Dinner,4,high,29.141722


### By group processing

In addition to aggregation, pandas `groupby` can be used to
replicate most other `bysort` processing from Stata. For example,
the following example lists the first observation in the current
sort order by sex/smoker group.

```stata
bysort sex smoker: list if _n == 1
```


In pandas this would be written as:

In [32]:
tips.groupby(["sex", "smoker"]).first()

total_bill   tip   day    time  size bucket  adj_total_bill
sex    smoker                                                             
Female No            5.25  1.00   Sat  Dinner     1    low      -11.938278
       Yes           1.07  1.00   Sat  Dinner     1    low      -17.686344
Male   No            5.51  2.00  Thur   Lunch     2    low      -11.678278
       Yes           5.25  5.15   Sun  Dinner     2    low      -13.506344

## Other considerations

### Disk vs memory

pandas and Stata both operate exclusively in memory. This means that the size of
data able to be loaded in pandas is limited by your machine’s memory.
If out of core processing is needed, one possibility is the
[dask.dataframe](https://dask.pydata.org/en/latest/dataframe.html)
library, which provides a subset of pandas functionality for an
on-disk `DataFrame`.